In [3]:
import pandas as pd

# Dosya yolunu ayarla
xlsx_path = "/content/dataaa.xlsx"
excel_file = pd.ExcelFile(xlsx_path)

# Hedef sütunlar
target_cols = ['ders_adi', 'dersin_amaci', 'dersin_icerigi', 'ders_ogrenim_ciktilari', 'haftalik_konular']

# Yeni dosya yazımı için hazırlık
with pd.ExcelWriter("/content/dataaa_with_havuz_tr.xlsx", engine="openpyxl", mode="w") as writer:
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(xlsx_path, sheet_name=sheet_name)

        # Eksik sütunları tamamla
        for col in target_cols:
            if col not in df.columns:
                df[col] = ""

        # havuz_tr sütununu oluştur
        df["havuz_tr"] = df[target_cols].fillna("").agg(" ".join, axis=1)

        # Yeni sayfayı yaz
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Yeni Excel dosyası oluşturuldu: dataaa_with_havuz_tr.xlsx")


Yeni Excel dosyası oluşturuldu: dataaa_with_havuz_tr.xlsx


In [6]:
import pandas as pd

# Excel dosyasını oku
df = pd.read_excel("/content/dataaa_with_havuz_tr.xlsx", sheet_name="MESLEKİ A")  # Sayfa1 yerine doğru sayfa adını yaz

# havuz_tr sütununu yazdır
print(df["havuz_tr"][0])


Dijital Pazarlama Bu ders, dijital pazarlama stratejileri ve araçlarını tanıtarak, öğrencilerin dijital platformlarda etkili pazarlama teknikleri geliştirme becerisi kazanmalarını sağlamayı amaçlar. Öğrenciler, dijital pazarlama kavramlarını öğrenir, sosyal medya, arama motoru optimizasyonu ve veri analitiği gibi araçları kullanarak stratejiler oluşturabilir. Dijital Pazarlama” dersi sonunda öğrenciler, dijital pazarlamanın temel kavramlarını ve stratejilerini açıklayabilecek, sosyal medya, arama motoru optimizasyonu (SEO) ve dijital kampanya araçlarını etkin şekilde kullanabilecektir. Hedef kitle analizi yapma, kullanıcı davranışlarını yorumlama ve dijital platformlarda etkili pazarlama planları geliştirme yetkinliği kazanacaklardır. Dersin ilk haftalarında dijital pazarlamanın temel kavramları, tarihsel gelişimi ve geleneksel pazarlamadan farkları ele alınır. Ardından dijital pazarlama stratejileri, hedef kitle belirleme, persona oluşturma ve kullanıcı yolculuğu haritalandırma gibi t

In [7]:
!pip install deep-translator


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.8 MB/s eta 0:00:00


✅ 2. Kod: Her sayfada havuz_tr sütununu İngilizceye çevir ve havuz_eng olarak ekle

In [8]:
from deep_translator import GoogleTranslator
import pandas as pd

# Excel dosyasını yükle
excel_path = "/content/dataaa_with_havuz_tr.xlsx"
excel_file = pd.ExcelFile(excel_path)

# Çevrilen dosyaları burada saklayacağız
updated_sheets = {}

# Her sayfa için işlemleri yap
for sheet_name in excel_file.sheet_names:
    df = pd.read_excel(excel_file, sheet_name=sheet_name)

    # Boş olmayan hücreleri İngilizceye çevir
    translations = []
    for text in df["havuz_tr"]:
        if pd.isna(text):
            translations.append("")
        else:
            try:
                translated = GoogleTranslator(source='tr', target='en').translate(text)
                translations.append(translated)
            except:
                translations.append("Translation Error")

    # Yeni sütun olarak ekle
    df["havuz_eng"] = translations

    # Güncellenmiş sayfayı sakla
    updated_sheets[sheet_name] = df

# Yeni Excel'e kaydet
with pd.ExcelWriter("/content/dataaa_with_havuz_tr_eng.xlsx") as writer:
    for sheet_name, df in updated_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("✅ Tüm sayfalarda İngilizce çeviri tamamlandı ve dosya kaydedildi.")


✅ Tüm sayfalarda İngilizce çeviri tamamlandı ve dosya kaydedildi.


✅ 1. Gerekli Kurulumlar

In [9]:
!pip install nltk
!python -m nltk.downloader stopwords


<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


✅ 2. Kod – Her sayfa için temizleme ve kaydetme işlemi

In [10]:
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords

# Stopword listeleri
stop_words_tr = set(stopwords.words('turkish'))
stop_words_en = set(stopwords.words('english'))

# Noktalama işaretleri
punct_table = str.maketrans('', '', string.punctuation)

# Temizleme fonksiyonu
def clean_text(text, lang="tr"):
    if pd.isna(text):
        return ""
    # Küçük harf, noktalama silme, kelimeye bölme
    words = text.lower().translate(punct_table).split()
    if lang == "tr":
        cleaned = [w for w in words if w not in stop_words_tr]
    else:
        cleaned = [w for w in words if w not in stop_words_en]
    return " ".join(cleaned)

# Dosyayı yükle
excel_path = "/content/dataaa_with_havuz_tr_eng.xlsx"
excel_file = pd.ExcelFile(excel_path)
updated_sheets = {}

# Her sayfa için işlem
for sheet_name in excel_file.sheet_names:
    df = pd.read_excel(excel_file, sheet_name=sheet_name)

    # Temizlenmiş sütunları oluştur
    df["havuz_tr_cleaned"] = df["havuz_tr"].apply(lambda x: clean_text(x, lang="tr"))
    df["havuz_eng_cleaned"] = df["havuz_eng"].apply(lambda x: clean_text(x, lang="en"))

    updated_sheets[sheet_name] = df

# Yeni dosya olarak kaydet
output_path = "/content/dataaa_with_cleaned_havuz.xlsx"
with pd.ExcelWriter(output_path) as writer:
    for sheet_name, df in updated_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("✅ Türkçe ve İngilizce stopword temizliği tamamlandı, dosya kaydedildi.")


✅ Türkçe ve İngilizce stopword temizliği tamamlandı, dosya kaydedildi.
